In [1]:
# importing the libraries
import numpy as np
import pandas as pd
import matplotlib
from glob import glob
from keras.models import Model
from keras.layers import Flatten, Dense
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from PyInquirer import prompt, print_json
from keras.models import model_from_json

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from tensorflow.python.keras.applications import ResNet50
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D, BatchNormalization
from tensorflow.python.keras.applications.resnet50 import preprocess_input
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.preprocessing.image import load_img, img_to_array

In [3]:
training_dir = '/visual/Semi_train'
test_dir = '/visual/Semi_test'

In [4]:
# useful for getting number of files
image_files = glob(training_dir + '/*/*.jp*g')
test_image_files = glob(test_dir + '/*/*.jp*g')

In [5]:
# getting the number of classes i.e. type of fruits
folders = glob(training_dir + '/*')
num_classes = len(folders)
print ('Total Classes = ' + str(num_classes))

Total Classes = 11


In [6]:
IMAGE_SIZE = [64, 64]  
training_datagen = ImageDataGenerator(horizontal_flip=True,
                                   width_shift_range = 0.4,
                                   height_shift_range = 0.4,
                                   zoom_range=0.3,
                                   rotation_range=20,
                                   )
test_datagen = ImageDataGenerator(horizontal_flip=True,
                                   width_shift_range = 0.4,
                                   height_shift_range = 0.4,
                                   zoom_range=0.3,
                                   rotation_range=20,
                                   )

training_generator = training_datagen.flow_from_directory(training_dir, target_size = IMAGE_SIZE, batch_size = 200, class_mode = 'categorical')
test_generator = test_datagen.flow_from_directory(test_dir, target_size = IMAGE_SIZE, batch_size = 200, class_mode= 'categorical')


Found 40264 images belonging to 11 classes.
Found 11876 images belonging to 11 classes.


In [7]:
# image_size = 224
# batch_size = 10
# train_generator = data_generator.flow_from_directory(
#         '../input/flowers-recognition/flowers/flowers/',
# #         target_size=(image_size, image_size),
#         batch_size=batch_size,
#         class_mode='categorical')

In [8]:
model = Sequential()

model.add(ResNet50(include_top=False, pooling='avg', weights = 'imagenet'))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(2048, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(num_classes, activation='softmax'))

model.layers[0].trainable = False

In [9]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:

model.fit_generator(training_generator,
                              steps_per_epoch = 201, 
                              epochs = 5,
                              validation_data = test_generator,
                              validation_steps= 59)


Epoch 1/5
201/201 [==============================] - 1402s 7s/step - loss: 1.8999 - acc: 0.4110 - val_loss: 2.3199 - val_acc: 0.3951
Epoch 2/5
201/201 [==============================] - 1372s 7s/step - loss: 1.5465 - acc: 0.4540 - val_loss: 2.2330 - val_acc: 0.4219
Epoch 3/5
201/201 [==============================] - 1406s 7s/step - loss: 1.5172 - acc: 0.4613 - val_loss: 2.2271 - val_acc: 0.4254
Epoch 4/5
201/201 [==============================] - 1371s 7s/step - loss: 1.4969 - acc: 0.4682 - val_loss: 2.2990 - val_acc: 0.4322
Epoch 5/5
201/201 [==============================] - 1420s 7s/step - loss: 1.4830 - acc: 0.4722 - val_loss: 2.2686 - val_acc: 0.4345


In [11]:
temp = image.load_img('/visual/Semi_test/Dress/194_comsumer_03.jpg',target_size=(64,64));

In [12]:
temp = image.img_to_array(temp)
temp = np.expand_dims(temp, axis=0)
result = model.predict(temp)


In [13]:
result

array([[2.1053301e-02, 3.0324402e-01, 3.6815446e-02, 4.1328077e-03,
        5.2279621e-01, 6.0268089e-02, 1.9336292e-04, 1.9746641e-02,
        2.7101650e-03, 1.1775832e-03, 2.7862413e-02]], dtype=float32)

In [14]:
training_generator.class_indices

{'Chiffon': 0,
 'Dress': 1,
 'Jeans': 2,
 'Lace_Dress': 3,
 'Lace_Shirt': 4,
 'Leggings_and_Jeans': 5,
 "Men's_Coats": 6,
 "Men's_Pants": 7,
 "Men's_Shirts": 8,
 "Men's_Summer_Wear": 9,
 "Men's_T_Shirt": 10}

In [16]:
model_json = model.to_json()
with open("/visual/Result/ResNet50-New.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")


Saved model to disk
